# Large problem with 1000 assets and 100 factors

In [2]:
import numpy as np
import cvxpy as cvx

from cvx.risk.factor.timeseries import TimeseriesFactorRiskModel
from util.random import random_factors, random_beta, random_noise, random_assets

In [3]:
T = 2000
factors = random_factors(T=T, N=100, const_factor=False)

In [4]:
beta = random_beta(assets=random_assets(1000), factors=factors)

In [5]:
ret = factors @ beta + 0.01 * random_noise(factors @ beta)

In [6]:
triangle = TimeseriesFactorRiskModel(factors=factors, returns=ret)

In [7]:
#w = triangle.weights
w = cvx.Variable(shape=1000)
objective = cvx.Minimize(triangle.variance(w))
constraints = [w >= 0, cvx.sum(w) == 1]
problem = cvx.Problem(objective=objective, constraints=constraints)
problem.solve()

print(triangle.variance(w).value)

1.0416963316537711e-07


In [8]:
w = cvx.Variable(shape=1000)
# formulation as used in external alpha...
objective = cvx.Minimize(cvx.sum_squares(triangle.variance_matrix @ w))
constraints = [w >= 0, cvx.sum(w) == 1]
problem = cvx.Problem(objective=objective, constraints=constraints)
problem.solve()

print(triangle.variance(w).value)

# check the solution
assert np.isclose(w.value.sum(), 1.0)
assert np.all(w.value > -0.01)

1.0561185016301142e-07
CPU times: user 995 ms, sys: 985 ms, total: 1.98 s
Wall time: 236 ms
